# Background
---

# Prior and Simulations
---

In [2]:
from glass_sim import *

## 1. Prior Parameters: $\bm{\theta}\sim P(\bm{\theta})$

In [13]:
def prior_param_samples(h_range, Oc_range, Ob_range, type, n_samples):
    assert isinstance(h_range, tuple) and len(h_range) == 2, "h_range must be a tuple of (min, max)"
    assert isinstance(Oc_range, tuple) and len(Oc_range) == 2, "Oc_range must be a tuple of (min, max)"
    assert isinstance(Ob_range, tuple) and len(Ob_range) == 2, "Ob_range must be a tuple of (min, max)"
    assert type in ["uniform", "normal"], "type must be either 'uniform' or 'normal'"
    assert isinstance(n_samples, int) and n_samples > 0, "n_samples must be a positive integer"
    if type == "uniform":
        h_samples = np.random.uniform(h_range[0], h_range[1], n_samples)
        Oc_samples = np.random.uniform(Oc_range[0], Oc_range[1], n_samples)
        Ob_samples = np.random.uniform(Ob_range[0], Ob_range[1], n_samples)
        samples = np.vstack((h_samples, Oc_samples, Ob_samples)).T
    elif type == "normal":
        cov = np.diag([h_range[1] - h_range[0], Oc_range[1] - Oc_range[0], Ob_range[1] - Ob_range[0]])**2 / 12
        mean = [(h_range[0] + h_range[1]) / 2, (Oc_range[0] + Oc_range[1]) / 2, (Ob_range[0] + Ob_range[1]) / 2]
        samples = np.random.multivariate_normal(mean, cov, n_samples)
    return samples


h_range = (0.6, 0.8)
Oc_range = (0.2, 0.4)
Ob_range = (0.03, 0.05)
n_samples = 1000
uniform_samples = prior_param_samples(h_range, Oc_range, Ob_range, "uniform", n_samples)
print("Uniform Samples:\n", uniform_samples)
normal_samples = prior_param_samples(h_range, Oc_range, Ob_range, "normal", n_samples)
print("Normal Samples:\n", normal_samples)


Uniform Samples:
 [[0.77376549 0.21837809 0.03078414]
 [0.72529927 0.37826868 0.03918482]
 [0.78668682 0.27350417 0.04972736]
 ...
 [0.71647135 0.35010276 0.03939765]
 [0.69206261 0.37843294 0.04189131]
 [0.71997984 0.34582458 0.0382579 ]]
Normal Samples:
 [[0.689695   0.28421681 0.03304922]
 [0.65800537 0.15767512 0.03825492]
 [0.65471729 0.17705651 0.04308088]
 ...
 [0.78054443 0.25200362 0.04095189]
 [0.78159793 0.33429954 0.0394831 ]
 [0.68977909 0.27827112 0.02927799]]


##  2. Simulated Data: $\bm{x}|\bm{\theta}$

In [ ]:
param_samples = []
cls_samples = []

for i in range(uniform_samples.shape[0]):
    cosmology = {}
    cosmology['h'], cosmology['Oc'], cosmology['Ob'] = uniform_samples[i] # unpacking
    sim = lensing_cls_sim(cosmology)
    param_samples.append(sim['params'])
    cls_samples.append(sim['cls'])
param_samples = np.array(param_samples)
cls_samples = np.array(cls_samples) 

print(param_samples)
print(cls_samples)

In [ ]:
import pickle
with open('sbi_demo_data.pkl', 'wb') as f:
    # pickle.dump({'params': param_samples, 'cls': cls_samples}, f)

# Compression
---

## Linear compression methods
Let $\{\mathbf{x}_1,...,\mathbf{x}_N\}$ be the set of simulated data with corresponding.  
$$
\tilde{\mathbf{x}}_{m\times1} = T\;\mathbf{x}_{n\times1}
$$
$T$ can be obtained via:

1. Canoncical correlation analysis
    - Maximises the correlation between the parameters and data (also could be written as maximising the mutual information).
    - Use canonical data vectors as compressed data
1. MOPED
    - Assumes Gaussian likelihood surface and preserves the Fisher Information, as a result:
        - requires fiducial cosmology simulations for covariance estimation
        - requires additional simulation cost for mean derivative estimation
1. e-MOPED 
    - Park, M., Gatti, M., & Jain, B. (2025). Dimensionality reduction techniques for statistical inference in cosmology.
    - Define new data space as a transform of the parameter space centred at the mean parameter.


In [ ]:
def cca(param_samples, cls_samples):
    '''
    Compute the sampled parameter auto covariance, simulated data vector auto covariance and the parameter-data vector cross covariance
    Methodology as per Park, M., Gatti, M., & Jain, B. (2025). Dimensionality reduction techniques for statistical inference in cosmology.
    '''
    cov = np.cov(params_samples.T, cls_samples.T)
    params_count = param_samples.shape[1]
    cp = cov[:params_count,:params_count]
    cd = cov[params_count:,params_count:]
    cpd = cov[:params_count,params_count:]

    cl = cpd.T@np.linalg.inv(cp)@cpd

    canon_corr, canon_vecs = scipy.linalg.eigh(cd, cd - cl)
    
    return {'params': params_samples, 'compressed_cls': cla_samples@canon_vecs}

In [ ]:
def moped(param_samples, cls_samples, fiducial_cls_samples, perturbed_param_delta, perturbed_cls_samples):
    '''
    fiducial_cls used for covariance computation
    perturbed_cls used for mean derivative computation
    '''
    cov_matrix = np.cov(fiducial_cls_samples.T)

    deriv = []
    for delta in perturbed_param_delta:
        deriv.append(np.mean(perturbed_cls_samples - fiducial_cls_samples, axis=0)/delta) #first order finite difference

    compression_matrix = np.zeros((param_samples.shape[1], cls_samples.shape[1])) # initialize compression matrix d_params x d_cls

    for i in range(param_samples.shape[1]):
        invcov_deriv = np.linalg.solve(cov_matrix, deriv[i])
        if i>0:
            coefs = transf_matrix[:i,:]@deriv[:i]
            transf_matrix[i,:]= (invcov_deriv - transf_matrix[:i,:].T@coefs)/np.sqrt(deriv[i]@invcov_deriv- np.sum(coefs**2))
        else:
            transf_matrix[i,:]= (invcov_deriv)/np.sqrt(deriv[i]@invcov_deriv)

    compressed_cls = compression_matrix@cls_samples.T
    
    return {'params': param_samples, 'compressed_cls': compressed_cls}


In [21]:
# Generate fiducial data for MOPED compression
fiducial_params = {'h': 0.7, 'Oc': 0.3, 'Ob': 0.04}
n_fiducial_samples = 80

fiducial_param_samples = []
fiducial_cls_samples = []
for n in range(n_fiducial_samples):
    sim = lensing_cls_sim(fiducial_params)
    fiducial_param_samples.append(sim['params'])
    fiducial_cls_samples.append(sim['cls'])
fiducial_param_samples = np.array(fiducial_param_samples)
fiducial_cls_samples = np.array(fiducial_cls_samples)

# print("Fiducial Params:\n", fiducial_param_samples)
# print("Fiducial Cls:\n", fiducial_cls_samples)

with open('moped_fiducial_cls.pkl', 'wb') as f:
    pickle.dump({'params': fiducial_param_samples, 'cls': fiducial_cls_samples}, f)

16it [00:00, 35.06it/s]
16it [00:00, 37.21it/s]
16it [00:00, 35.32it/s]
16it [00:00, 37.62it/s]
16it [00:00, 37.11it/s]
16it [00:00, 36.24it/s]
16it [00:00, 37.22it/s]
16it [00:00, 36.07it/s]
16it [00:00, 35.90it/s]
16it [00:00, 36.23it/s]
16it [00:00, 35.85it/s]
16it [00:00, 36.28it/s]
16it [00:00, 36.87it/s]
16it [00:00, 37.09it/s]
16it [00:00, 36.64it/s]
16it [00:00, 35.46it/s]
16it [00:00, 35.23it/s]
16it [00:00, 34.44it/s]
16it [00:00, 34.56it/s]
16it [00:00, 37.21it/s]
16it [00:00, 37.09it/s]
16it [00:00, 37.12it/s]
16it [00:00, 36.15it/s]
16it [00:00, 32.73it/s]
16it [00:00, 32.78it/s]
16it [00:00, 35.10it/s]
16it [00:00, 33.10it/s]
16it [00:00, 37.44it/s]
16it [00:00, 37.89it/s]
16it [00:00, 37.06it/s]
16it [00:00, 35.98it/s]
16it [00:00, 33.18it/s]
16it [00:00, 37.15it/s]
16it [00:00, 33.12it/s]
16it [00:00, 33.39it/s]
16it [00:00, 36.37it/s]
16it [00:00, 36.96it/s]
16it [00:00, 35.43it/s]
16it [00:00, 36.42it/s]
16it [00:00, 33.31it/s]
16it [00:00, 33.25it/s]
16it [00:00, 36.

In [ ]:
def emoped(param_samples, cls_samples, fiducial_cls, perturbed_cls):
    '''
    fiducial_cls used for covariance computation
    perturbed_cls used for mean derivative computation
    '''
    return {'params': param_samples, 'compressed_cls': compressed_cls}

# Density Estimation
---